In [1]:
import pandas as pd

# Load the two datasets
prolific_df = pd.read_excel("Prolific_Data_All_with_time.xlsx", sheet_name="Sheet1")
negative_low_df = pd.read_excel("Negative_Low_with_time.xlsx", sheet_name="Sheet1")

# Append / concatenate the two datasets
df = pd.concat([prolific_df, negative_low_df], ignore_index=True)


In [2]:
# Drop if subject didnt complete the experiment -->'Earning' is missing
df = df[df['Earning'].notna()]

# Drop subjects who returned their submission based on Prolific_ID
returners = [
    "61549a2caa92d84f392000e9", "5ec62902bb44c16b842128f9", "56b0ac5fe8b677000cdc0a34",
    "65dd365163f7579fba938dea", "5c2fcd716ea6880001dc8e3d", "63e51c6b462cfa8a42eba8b2",
    "6571af52592a29d0854f366e", "6234d77291179c1badffad36", "63c09186d75a9bbd3bec5652",
    "5ffa5b733bd20c42d0b40b87", "5ced23c585712e00190bd98a", "66154557c058526ce9f1cfe2"
]

df = df[~df['Prolific_ID'].isin(returners)]


In [3]:
# Filter non-investors and those not fluent in English
df = df[(df['Fin_market'] != 2) & (df['English'] != 2)]

# Drop participants who made 3 mistakes and failed the attention check
df = df[~((df['correct_count'] == 3) & (df['P2_AttenCheck_2'] != 5))]

# Create a participant ID 
df['id'] = range(1, len(df) + 1)

# Map treatment conditions to numeric codes
treatment_map = {
    'Baseline': 1,
    'Positive_Higher': 2,
    'Negative': 3,
    'Positive_Lower': 4
}
df['treatment_num'] = df['treatment'].map(treatment_map)

# Drop "Positive_Lower" condition (treatment_num == 4) as we do not use it in the main analyses
df = df[df['treatment_num'] != 4]


In [4]:
# Recode Gender: 1 -> 0, 2 -> 1, 3/4 -> 0
df['Gender'] = df['Gender'].replace({1: 0, 2: 1, 3: 0, 4: 0})

# Recode Intere_inves and Own_asset: 2 -> 0
df['Intere_inves'] = df['Intere_inves'].replace({2: 0})
df['Own_asset'] = df['Own_asset'].replace({2: 0})

# Trade_frequently: 1 if Trade_freq < 4
df['Trade_frequently'] = (df['Trade_freq'] < 4).astype(int)

# High_wealth: 1 if Inve_amount >= 4 and not 7 (7 is "don't know")
df['High_wealth'] = ((df['Inve_amount'] >= 4) & (df['Inve_amount'] != 7)).astype(int)

# High_CurrentSustainShare: 1 if Sustain_share >= 4 and not 7
df['High_CurrentSustainShare'] = ((df['Sustain_share'] >= 4) & (df['Sustain_share'] != 7)).astype(int)

# Invest_in_SRI: 1 if Sustain_share != 1 and != 7
df['Invest_in_SRI'] = ((df['Sustain_share'] != 1) & (df['Sustain_share'] != 7)).astype(int)


In [5]:
# Standardize Share_willing and Donate_charity
df['Share_willing_std'] = (df['Share_willing'] - df['Share_willing'].mean()) / df['Share_willing'].std()
df['Donate_charity_std'] = (df['Donate_charity'] - df['Donate_charity'].mean()) / df['Donate_charity'].std()

# Compute the Falk et al. (2023)-based Social Preferences measure
df['Social_Preferences'] = (
    0.635 * df['Share_willing_std'] +
    0.365 * df['Donate_charity_std']
)


In [6]:
# Define optimal share allocation benchmarks based on performed optimization calculations 
df['HighCorr_OptimalShare_lowreturn'] = 23.90
df['LowCorr_OptimalShare_lowreturn'] = 45.60


In [7]:
# Environmental Literacy Questions
df['correct_Environ_Literacy1'] = (df['Environ_Literacy1'] == 2).astype(int)
df['correct_Environ_Literacy2'] = (df['Environ_Literacy2'] == 2).astype(int)
df['correct_Environ_Literacy3'] = (df['Environ_Literacy3'] == 1).astype(int)
df['correct_Environ_Literacy4'] = (df['Environ_Literacy4'] == 1).astype(int)
df['correct_Environ_Literacy5'] = (df['Environ_Literacy5'] == 4).astype(int)

# Total Environmental Literacy Score
df['correct_Environ_Literacy'] = (
    df['correct_Environ_Literacy1'] +
    df['correct_Environ_Literacy2'] +
    df['correct_Environ_Literacy3'] +
    df['correct_Environ_Literacy4'] +
    df['correct_Environ_Literacy5']
)

# Financial Literacy Questions
df['correct_Fin_lite_Q1'] = (df['Fin_lite_Q1'] == 2).astype(int)
df['correct_Fin_lite_Q2'] = (df['Fin_lite_Q2'] == 1).astype(int)
df['correct_Fin_lite_Q3'] = (df['Fin_lite_Q3'] == 3).astype(int)
df['correct_Fin_lite_Q4'] = (df['Fin_lite_Q4'] == 2).astype(int)

# Financial literacy without Q1 (as in lusardi and mitchell (2014) financial literacy score)
df['numcorrect_financialliteracy'] = (
    df['correct_Fin_lite_Q2'] +
    df['correct_Fin_lite_Q3'] +
    df['correct_Fin_lite_Q4']
)

# Financial literacy including all questions
df['numcorrect_financialliteracyall'] = (
    df['correct_Fin_lite_Q1'] +
    df['correct_Fin_lite_Q2'] +
    df['correct_Fin_lite_Q3'] +
    df['correct_Fin_lite_Q4']
)


In [8]:
# Comprehension Questions: correct answers
df['correct_PosiRela1'] = (df['PosiRela1'] == "drop").astype(int)
df['correct_PosiRela2'] = (df['PosiRela2'] == "rise").astype(int)
df['correct_NegaRela1'] = (df['NegaRela1'] == "rise").astype(int)
df['correct_NegaRela2'] = (df['NegaRela2'] == "drop").astype(int)

# Total comprehension score
df['correct_comprehension'] = (
    df['correct_PosiRela1'] +
    df['correct_PosiRela2'] +
    df['correct_NegaRela1'] +
    df['correct_NegaRela2']
)

# Drop participants who got 0 comprehension questions correctly to ensure quality responses 
df = df[df['correct_comprehension'] > 0]


In [9]:
# P1 investment asset by return level
df['P1_investmentAsset_highreturn'] = df['P1_investmentAsset'].where(df['P1_high_return'] == 1, df['P1_investmentAsset2'])
df['P1_investmentAsset_lowreturn'] = df['P1_investmentAsset'].where(df['P1_high_return'] == 0, df['P1_investmentAsset2'])

# P2 investment asset by return level
df['P2_investmentAsset_highreturn'] = df['P2_investmentAsset'].where(df['P1_high_return'] == 1, df['P2_investmentAsset2'])
df['P2_investmentAsset_lowreturn'] = df['P2_investmentAsset'].where(df['P1_high_return'] == 0, df['P2_investmentAsset2'])

# Assets assigned to correlation order
df['LowCorr_investmentAsset_highret'] = df['P1_investmentAsset_highreturn'].where(df['P1_low_first'] == 1, df['P2_investmentAsset_highreturn'])
df['HighCorr_investmentAsset_highret'] = df['P1_investmentAsset_highreturn'].where(df['P1_low_first'] == 0, df['P2_investmentAsset_highreturn'])

df['LowCorr_investmentAsset_lowret'] = df['P1_investmentAsset_lowreturn'].where(df['P1_low_first'] == 1, df['P2_investmentAsset_lowreturn'])
df['HighCorr_investmentAsset_lowret'] = df['P1_investmentAsset_lowreturn'].where(df['P1_low_first'] == 0, df['P2_investmentAsset_lowreturn'])


In [10]:
# Convert allocations to percentages
df['LowCorr_investmentAsset_highret'] = (df['LowCorr_investmentAsset_highret'] / 10000) * 100
df['LowCorr_investmentAsset_lowret'] = (df['LowCorr_investmentAsset_lowret'] / 10000) * 100

df['HighCorr_investmentAsset_highret'] = (df['HighCorr_investmentAsset_highret'] / 10000) * 100
df['HighCorr_investmentAsset_lowret'] = (df['HighCorr_investmentAsset_lowret'] / 10000) * 100

# Define key variables for analysis
df['correlation_neglect'] = df['HighCorr_investmentAsset_lowret'] - df['LowCorr_investmentAsset_lowret']
df['consider_corr'] = df['LowCorr_investmentAsset_lowret'] - df['HighCorr_investmentAsset_lowret']


In [14]:
# Convert beliefs from dollar to percent for more intuitive interpretation
belief_vars = [
    'P1_belief_upper', 'P1_belief_lower', 'P1_Expect_value',
    'P2_belief_upper', 'P2_belief_lower', 'P2_Expect_value'
]

for var in belief_vars:
    df[var] = (df[var] - 10000) / 100


In [18]:
# Perception of dependence between assets' returns
# Relationship perception (probabilities)
df['LowCorr_Relationship4'] = df['P1_Relationship4'].where(df['P1_low_first'] == 1, df['P2_Relationship4_2'])
df['HighCorr_Relationship4'] = df['P1_Relationship4'].where(df['P1_low_first'] == 0, df['P2_Relationship4_2'])

df['LowCorr_Relationship5'] = df['P1_Relationship5'].where(df['P1_low_first'] == 1, df['P2_Relationship5_2'])
df['HighCorr_Relationship5'] = df['P1_Relationship5'].where(df['P1_low_first'] == 0, df['P2_Relationship5_2'])

# Directional correctness
df['P1_Relationship2_'] = (
    ((df['P1_Relationship2'] == 'rise') & (df['P1_low_first'] == 1)) |
    ((df['P1_Relationship2'] == 'drop') & (df['P1_low_first'] == 0))
).astype(int)

df['P1_Relationship3_'] = (
    ((df['P1_Relationship3'] == 'drop') & (df['P1_low_first'] == 1)) |
    ((df['P1_Relationship3'] == 'rise') & (df['P1_low_first'] == 0))
).astype(int)

df['P2_Relationship2'] = (
    ((df['P2_Relationship2_2'] == 'drop') & (df['P1_low_first'] == 1)) |
    ((df['P2_Relationship2_2'] == 'rise') & (df['P1_low_first'] == 0))
).astype(int)

df['P2_Relationship3'] = (
    ((df['P2_Relationship3_2'] == 'drop') & (df['P1_low_first'] == 0)) |
    ((df['P2_Relationship3_2'] == 'rise') & (df['P1_low_first'] == 1))
).astype(int)

# Total correct answers for perceived dependence
df['correct_dependence'] = (
    df['P1_Relationship2_'] +
    df['P1_Relationship3_'] +
    df['P2_Relationship2'] +
    df['P2_Relationship3']
)


In [19]:
# Duplicate each row for round-based reshaping
df_round1 = df.copy()
df_round1['First_Round'] = 1
df_round1['Share_lowreturn'] = df_round1['P1_investmentAsset_lowreturn']
df_round1['Expect_value'] = df_round1['P1_Expect_value']
df_round1['Lower_Bound'] = df_round1['P1_belief_lower']
df_round1['Upper_Bound'] = df_round1['P1_belief_upper']

df_round2 = df.copy()
df_round2['First_Round'] = 0
df_round2['Share_lowreturn'] = df_round2['P2_investmentAsset_lowreturn']
df_round2['Expect_value'] = df_round2['P2_Expect_value']
df_round2['Lower_Bound'] = df_round2['P2_belief_lower']
df_round2['Upper_Bound'] = df_round2['P2_belief_upper']

# Combine into one dataframe
df_long = pd.concat([df_round1, df_round2], ignore_index=True)

# Define correlation type based on order and round
df_long['High_Corr'] = ((df_long['First_Round'] == 1) & (df_long['P1_low_first'] == 0)) | \
                       ((df_long['First_Round'] == 0) & (df_long['P1_low_first'] == 1))
df_long['High_Corr'] = df_long['High_Corr'].astype(int)

df_long['Low_Corr'] = 1 - df_long['High_Corr']

# Convert Share_lowreturn to percentage
df_long['Share_lowreturn'] = (df_long['Share_lowreturn'] / 10000) * 100


In [20]:
#Assign Belief Variables (Return Expectations & Bounds) to Asset Correlation
# Expected return
df['LowCorr_Expect_value'] = df['P1_Expect_value'].where(df['P1_low_first'] == 1, df['P2_Expect_value'])
df['HighCorr_Expect_value'] = df['P1_Expect_value'].where(df['P1_low_first'] == 0, df['P2_Expect_value'])

df['Avg_Expect_value'] = (df['LowCorr_Expect_value'] + df['HighCorr_Expect_value']) / 2
df['Diff_Expect_value'] = df['LowCorr_Expect_value'] - df['HighCorr_Expect_value']

# Lower bound
df['LowCorr_belief_lower'] = df['P1_belief_lower'].where(df['P1_low_first'] == 1, df['P2_belief_lower'])
df['HighCorr_belief_lower'] = df['P1_belief_lower'].where(df['P1_low_first'] == 0, df['P2_belief_lower'])

df['Avg_belief_lower'] = (df['LowCorr_belief_lower'] + df['HighCorr_belief_lower']) / 2
df['Diff_belief_lower'] = df['LowCorr_belief_lower'] - df['HighCorr_belief_lower']

# Upper bound
df['LowCorr_belief_upper'] = df['P1_belief_upper'].where(df['P1_low_first'] == 1, df['P2_belief_upper'])
df['HighCorr_belief_upper'] = df['P1_belief_upper'].where(df['P1_low_first'] == 0, df['P2_belief_upper'])

df['Avg_belief_upper'] = (df['LowCorr_belief_upper'] + df['HighCorr_belief_upper']) / 2
df['Diff_belief_upper'] = df['LowCorr_belief_upper'] - df['HighCorr_belief_upper']


In [21]:
# Return perception: gain/loss probabilities (Q1–Q3)
df['LowCorr_Return_perception1'] = df['P1_Return_perception1'].where(df['P1_low_first'] == 1, df['P2_Return_perception1'])
df['HighCorr_Return_perception1'] = df['P1_Return_perception1'].where(df['P1_low_first'] == 0, df['P2_Return_perception1'])

df['LowCorr_Return_perception2'] = df['P1_Return_perception2'].where(df['P1_low_first'] == 1, df['P2_Return_perception2'])
df['HighCorr_Return_perception2'] = df['P1_Return_perception2'].where(df['P1_low_first'] == 0, df['P2_Return_perception2'])

df['LowCorr_Return_perception3'] = df['P1_Return_perception3'].where(df['P1_low_first'] == 1, df['P2_Return_perception3'])
df['HighCorr_Return_perception3'] = df['P1_Return_perception3'].where(df['P1_low_first'] == 0, df['P2_Return_perception3'])

# Averages
df['Avg_Return_perception1'] = (df['LowCorr_Return_perception1'] + df['HighCorr_Return_perception1']) / 2
df['Avg_Return_perception2'] = (df['LowCorr_Return_perception2'] + df['HighCorr_Return_perception2']) / 2
df['Avg_Return_perception3'] = (df['LowCorr_Return_perception3'] + df['HighCorr_Return_perception3']) / 2


In [22]:
# Subjective risk perception
df['LowCorr_Risk_perception1'] = df['P1_Risk_perception1'].where(df['P1_low_first'] == 1, df['P2_Risk_perception1'])
df['HighCorr_Risk_perception1'] = df['P1_Risk_perception1'].where(df['P1_low_first'] == 0, df['P2_Risk_perception1'])


In [23]:
# Duplicate each row to create First_Round/Second_Round rows
df_expanded = pd.concat([df.assign(dup=0), df.assign(dup=1)], ignore_index=True)

# Mark round
df_expanded['First_Round'] = (df_expanded['dup'] == 0).astype(int)

# Asset order indicators
df_expanded['High_Corr'] = df_expanded.apply(
    lambda row: 1 if (row['First_Round'] == 1 and row['P1_low_first'] == 0) or
                     (row['First_Round'] == 0 and row['P1_low_first'] == 1) else 0,
    axis=1
)
df_expanded['Low_Corr'] = 1 - df_expanded['High_Corr']

# Share invested in low-return asset (converted to %)
df_expanded['Share_lowreturn'] = df_expanded.apply(
    lambda row: row['P1_investmentAsset_lowreturn'] if row['First_Round'] == 1 else row['P2_investmentAsset_lowreturn'],
    axis=1
)
df_expanded['Share_lowreturn'] = df_expanded['Share_lowreturn'] / 10000 * 100


In [27]:
# Translate allocation shares to percentages
df['LowCorr_investmentAsset_lowret'] = df['LowCorr_investmentAsset_lowret'] / 10000 * 100
df['HighCorr_investmentAsset_lowret'] = df['HighCorr_investmentAsset_lowret'] / 10000 * 100

# Correlation neglect and consideration
df['correlation_neglect'] = df['HighCorr_investmentAsset_lowret'] - df['LowCorr_investmentAsset_lowret']
df['consider_corr'] = df['LowCorr_investmentAsset_lowret'] - df['HighCorr_investmentAsset_lowret']


In [30]:
# Create a dictionary of labels
var_labels = {
    'Own_asset': "Owns Equity",
    'Intere_inves': "Interested Financial Markets",
    'Trade_frequently': "Frequent Trader",
    'High_wealth': "High Wealth",
    'High_CurrentSustainShare': "High SRI Share",
    'Invest_in_SRI': "Invest in SRI",
    'Risk_taking': "Risk Preferences",
    'Share_willing': "Willingness to Share",
    'Donate_charity': "Hypothetical Donation",
    'Social_Preferences': "Social Preferences",
    'Donation': "Incentivized Donation",
    'correct_Environ_Literacy': "Environmental Literacy",
    'Sustainability_Value': "Incentivized Sustainable Investment",
    'numcorrect_financialliteracyall': "Financial Literacy",
    'correct_comprehension': "Correlation Comprehension",
    'Stat_know': "Statistics Knowledge",
    'Number_think': "Doesn't Think about Numbers",
    'Number_inter': "Consider Numbers in Decisions",
    'P1_low_first': "Correlation Order",
    'P1_high_return': "Asset Order"
}


In [33]:
df.to_csv("transformed_data.csv", index=False)